In [1]:
import math
import pandas as pd
import json

In [2]:
def get_children_in_depth(df, depth, key, children_in_depth_dict):
    if not children_in_depth_dict[key][depth]:
        return depth - 1

    children_from_current_depth_list = []
    for children in children_in_depth_dict[key][depth]:
        children_from_current_depth_list += list(df[df['Parent_Name'] == children]['Name'].values)

    children_in_depth_dict[key][depth + 1] = children_from_current_depth_list
    return get_children_in_depth(df, depth + 1, key, children_in_depth_dict)